In [1]:
import pandas as pd 
import glob
import cv2 as cv
import random
import os
import numpy as np
import random
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from tqdm.auto import tqdm
import timm

def pred_distance(review_img_path, product_img_path):
    df = pd.DataFrame(columns=['review_img_path','product_img_path', 'label'])
    df['review_img_path'] = [review_img_path]
    df['product_img_path'] = [product_img_path]
    df['label'] = [0]
    
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    
    CFG = {
        'IMG_SIZE':224,
        'EPOCHS':1,
        'LEARNING_RATE':3e-4,
        # 'LEARNING_RATE':10,
        'BATCH_SIZE':1,
        'SEED':41
    }
    
    class SiameseNetworkDataset(Dataset):
        def __init__(self,review_img_path,product_img_path,label,transform=None):
            self.review_img_path = review_img_path
            self.product_img_path = product_img_path
            self.label = label
            self.transform = transform

        def __getitem__(self,index):
            review_img = cv.imread(self.review_img_path[index])
            product_img = cv.imread(self.product_img_path[index])
            review_img = cv.resize(review_img, (CFG['IMG_SIZE'], CFG['IMG_SIZE']))
            product_img = cv.resize(product_img, (CFG['IMG_SIZE'], CFG['IMG_SIZE']))

            if self.transform is not None:
                review_img  = self.transform(image=review_img)['image']
                product_img  = self.transform(image=product_img)['image']

            return review_img, product_img, self.label[index]

        def __len__(self):
            return len(self.review_img_path)
        
    train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            ToTensorV2()
                            ])
    
    val_dataset = SiameseNetworkDataset(df["review_img_path"].values, df["product_img_path"].values, df["label"].values, train_transform)
    val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

    class BaseModel(nn.Module):
        def __init__(self):
            super(BaseModel, self).__init__()
            self.backbone = timm.create_model('efficientnet_b0', pretrained=False)
            self.classifier = nn.Linear(1000, 50)
            self.dropout = nn.Dropout(0.1)
            self.ReLU = nn.ReLU(inplace=False)

        def forward(self, x, y):
            x = self.backbone(x)
            x = self.classifier(x)

            y = self.backbone(y)
            y = self.classifier(y)

            z = F.pairwise_distance(x, y, keepdim = True)
            return z
        
    def validation(model, val_loader, device):
        model.eval()
        pred_list = []
        
        with torch.no_grad():
            for review_img, product_img, labels in tqdm(iter(val_loader)):
                review_img = review_img.float().to(device)
                product_img = product_img.float().to(device)
    
                
                pred = model(review_img, product_img)
                pred = pred.detach().cpu().numpy().tolist()
                pred_list += pred
            
        return pred_list 
        
    def train(model,  val_loader,  device):
        model = model.to(device)

        model.train()
        prediction = validation(model, val_loader, device)

        return prediction
        
    
    model = BaseModel()
    model.load_state_dict(torch.load('./distance_VGGBase_E_Contra.pt'))
    model.eval()

    prediction = train(model,  val_loader, device)
    
    return prediction[0][0]
    
print(pred_distance("./masked_data/review_img/0_review_img/O/404.jpg", "./masked_data/product_img/0.jpg" ))

  0%|          | 0/1 [00:00<?, ?it/s]

0.14068768918514252


---

In [21]:
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

prediction, prediction_list = train(model, optimizer, None, val_loader, scheduler, device)

  0%|          | 0/1 [00:00<?, ?it/s]

1.0
0.0
0.0
0.0
Epoch [0], Train Loss : [nan] Val Loss : [0.01979] Val accuracy score : [100.00000]


/opt/conda/envs/SAM/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/conda/envs/SAM/lib/python3.9/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [22]:
prediction[0]

[0.14068756997585297]

In [23]:
prediction_list

[0]